<a href="https://colab.research.google.com/github/Nicolenki7/IBM-HR-Analytics-Employee-Attrition-Performance/blob/main/IBM_HR_Analytics_Employee_Attrition_%26_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

# 1. Cargar el Dataset
# Asumimos que el archivo 'HR.csv' ya está cargado en el entorno de Colab
df = pd.read_csv('HR.csv')

print("--- 1. Carga Inicial ---")
print(f"Dimensiones iniciales: {df.shape}")
print("-" * 30)

# 2. Manejo de la Columna Problemática (TotalCharges)
# TotalCharges a menudo contiene espacios vacíos (' ') que impiden su conversión a numérico.
# Paso A: Reemplazar espacios vacíos con NaN
df['TotalWorkingYears'] = df['TotalWorkingYears'].replace(' ', np.nan)
df['TotalWorkingYears'] = df['TotalWorkingYears'].replace(0, np.nan) # Asumiendo que 0 años es un error o NaN

# Paso B: Convertir la columna a tipo flotante (float)
# Usamos errors='coerce' para convertir cualquier valor no válido restante a NaN
df['TotalWorkingYears'] = pd.to_numeric(df['TotalWorkingYears'], errors='coerce')

# 3. Manejo de Valores Faltantes (Imputación Simple)
# Solo existen unos pocos valores faltantes en TotalCharges/TotalWorkingYears. Los imputaremos con la media.
# Esto asegura que no perdamos filas, manteniendo la integridad del análisis predictivo.

# Rellenar los valores faltantes (NaN) en TotalWorkingYears con la media de la columna
df['TotalWorkingYears'].fillna(
    df['TotalWorkingYears'].mean(),
    inplace=True
)

# 4. Preparación de la Variable Objetivo (Codificación Binaria)
# La variable 'Attrition' (Abandono) debe ser numérica para el modelo de Regresión Logística.
# 'Yes' -> 1 (Caso de interés)
# 'No' -> 0 (Caso base)
df['Attrition_Numeric'] = df['Attrition'].apply(
    lambda x: 1 if x == 'Yes' else 0
)

# 5. Verificación Rápida y Limpieza Final
# Eliminamos columnas constantes o redundantes que no aportan al modelo
df.drop(columns=['EmployeeCount', 'StandardHours', 'EmployeeNumber', 'Attrition', 'Over18'], inplace=True, errors='ignore')

print("--- 2. Limpieza y Codificación Completada ---")
print("Verificación de la Variable Objetivo:")
print(df['Attrition_Numeric'].value_counts())
print("-" * 30)
print("Verificación de valores faltantes (Debe ser 0):")
print(df['TotalWorkingYears'].isnull().sum())

# Mostrar las primeras filas del DataFrame limpio
print("\nDataFrame Limpio y Listo (df_clean):")
df_clean = df.copy() # Creamos una copia para el siguiente paso
print(df_clean.head())

--- 1. Carga Inicial ---
Dimensiones iniciales: (1470, 35)
------------------------------
--- 2. Limpieza y Codificación Completada ---
Verificación de la Variable Objetivo:
Attrition_Numeric
0    1470
Name: count, dtype: int64
------------------------------
Verificación de valores faltantes (Debe ser 0):
0

DataFrame Limpio y Listo (df_clean):
   Age     BusinessTravel  DailyRate              Department  \
0   41      Travel_Rarely       1102                   Sales   
1   49  Travel_Frequently        279  Research & Development   
2   37      Travel_Rarely       1373  Research & Development   
3   33  Travel_Frequently       1392  Research & Development   
4   27      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EnvironmentSatisfaction  \
0                 1          2  Life Sciences                        2   
1                 8          1  Life Sciences                        3   
2                 2          2          Other 

/tmp/ipython-input-1727104784.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalWorkingYears'].fillna(


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import sqlite3
import gc
import os

# 0. INICIALIZACIÓN GLOBAL
df_fe_ready = pd.DataFrame()
FILE_NAME = 'HR.csv'

# 1. CARGA DE DATOS, LIMPIEZA Y FEATURE ENGINEERING (CORRECCIÓN DE ATTRITION)
try:
    print(f"--- 1. Leyendo y Limpiando: {FILE_NAME} ---")

    df = pd.read_csv(FILE_NAME, skipinitialspace=True)

    # --- LIMPIEZA DE DATOS ---
    df['TotalWorkingYears'] = pd.to_numeric(df['TotalWorkingYears'], errors='coerce')
    df['TotalWorkingYears'].fillna(df['TotalWorkingYears'].mean(), inplace=True)

    # *** CORRECCIÓN CRÍTICA FINAL: Attrition ya está codificada como 0/1 ***
    # Simplemente la renombramos y aseguramos que sea INT.
    # Los valores 1 y 0 representan 'Yes' y 'No' respectivamente.
    df['Attrition_Numeric'] = pd.to_numeric(df['Attrition'], errors='coerce').fillna(0).astype(int)

    # Aseguramos que la columna 'Attrition' original (que podría ser de tipo object/string) se elimine
    df.drop(columns=['EmployeeCount', 'StandardHours', 'EmployeeNumber', 'Attrition', 'Over18'], inplace=True, errors='ignore')
    if 'Id' in df.columns:
        df.drop(columns=['Id'], inplace=True, errors='ignore')

    # Conversión explícita a tipo 'str' para categóricas (Solución a ValueError: 'Sales')
    for col in ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']:
        if col in df.columns:
            df[col] = df[col].astype(str)

    df_clean = df.copy()
    for col in ['Age', 'DailyRate', 'HourlyRate', 'Education']:
         if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(df_clean[col].median()).astype(int)

    del df
    gc.collect()

    # --- FEATURE ENGINEERING SQL ---
    conn = sqlite3.connect(':memory:')
    df_clean.to_sql('hr_data_clean', conn, if_exists='replace', index=False)
    sql_query_fe = """
    SELECT *,
        CASE WHEN TotalWorkingYears <= 3 THEN 'Entry_Level'
             WHEN TotalWorkingYears <= 8 THEN 'Mid_Level'
             WHEN TotalWorkingYears <= 15 THEN 'Senior_Level'
             ELSE 'Veteran_Level' END AS Seniority_Category,
        CASE WHEN MonthlyIncome < 3000 THEN 'Low_Income'
             WHEN MonthlyIncome < 7000 THEN 'Medium_Income'
             ELSE 'High_Income' END AS Monthly_Income_Level
    FROM hr_data_clean;
    """
    df_fe_ready = pd.read_sql_query(sql_query_fe, conn)
    conn.close()
    del df_clean
    gc.collect()
    print("Limpieza y Feature Engineering completados con éxito.")

except Exception as e:
    print(f"\nERROR DURANTE EL PROCESAMIENTO (Limpieza/SQL): {e}")
    df_fe_ready = pd.DataFrame()


# 2. MODELADO PREDICTIVO (CON SOLUCIONES HISTÓRICAS)
if df_fe_ready.empty:
    print("--- 2. Modelado omitido. El DataFrame final está vacío. ---")
else:
    print("--- 2. Iniciando Modelado Predictivo ---")

    y = df_fe_ready['Attrition_Numeric'].astype(int)
    X = df_fe_ready.drop(columns=['Attrition_Numeric'], errors='ignore')

    X.drop(columns=['index', 'Id'], errors='ignore', inplace=True)
    X.reset_index(drop=True, inplace=True)

    # 2.2. AISLAMIENTO Y CODIFICACIÓN DE TIPOS (Mantiene la solución a Value/TypeErrors)
    categorical_cols = X.select_dtypes(include=['object']).columns
    numerical_cols = X.select_dtypes(include=np.number).columns

    if not categorical_cols.empty:
        X_cat = pd.get_dummies(X[categorical_cols], drop_first=True)
    else:
        X_cat = pd.DataFrame()

    X_num = X[numerical_cols].copy()
    X_num = X_num.fillna(X_num.mean())

    X_encoded = pd.concat([X_num.reset_index(drop=True), X_cat.reset_index(drop=True)], axis=1)

    # *** VALIDACIÓN DE CLASES (Ahora SÍ deberían existir las dos clases) ***
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

    if y_train.nunique() < 2:
        print("\n!!! ERROR CRÍTICO: La variable objetivo (Attrition_Numeric) solo contiene una clase. !!!")
        print("La inspección sugiere que el archivo CSV tiene los valores 0 y 1. Por favor, verifique el archivo.")
        exit()

    # 2.3. Entrenamiento y Extracción de Drivers
    model = LogisticRegression(solver='liblinear', random_state=42, max_iter=200)
    model.fit(X_train, y_train)

    coefficients = pd.Series(model.coef_[0], index=X_train.columns)

    df_drivers = coefficients.to_frame('Coefficient').reset_index()
    df_drivers.columns = ['Feature', 'Coefficient']
    df_drivers['Abs_Coefficient'] = np.abs(df_drivers['Coefficient'])

    df_drivers = df_drivers.sort_values(by='Abs_Coefficient', ascending=False)

    df_drivers_top = df_drivers.head(15).copy()
    df_drivers_top['Coefficient'] = df_drivers_top['Coefficient'].round(4)

    # Resultados y Exportación
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, target_names=['No Abandono', 'Abandono'], output_dict=True)

    print("\n--- 3. Modelado Predictivo Completado con ÉXITO ---")
    print(f"Accuracy del Modelo: {report['accuracy']:.4f}")
    print("TOP 10 DRIVERS (Coeficientes):")
    print(df_drivers_top[['Feature', 'Coefficient']].head(10))

    df_drivers_top[['Feature', 'Coefficient']].to_csv('hr_churn_drivers.csv', index=False)
    df_fe_ready.to_csv('hr_data_fe_ready.csv', index=False)
    print("\nArchivos 'hr_churn_drivers.csv' y 'hr_data_fe_ready.csv' generados.")

--- 1. Leyendo y Limpiando: HR.csv ---


/tmp/ipython-input-1080301170.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalWorkingYears'].fillna(df['TotalWorkingYears'].mean(), inplace=True)


Limpieza y Feature Engineering completados con éxito.
--- 2. Iniciando Modelado Predictivo ---

--- 3. Modelado Predictivo Completado con ÉXITO ---
Accuracy del Modelo: 0.8810
TOP 10 DRIVERS (Coeficientes):
                               Feature  Coefficient
43                          OverTime_1       1.3729
23    BusinessTravel_Travel_Frequently       0.6666
34       JobRole_Laboratory Technician       0.5881
13                   PerformanceRating       0.5128
42                MaritalStatus_Single       0.4846
48  Monthly_Income_Level_Medium_Income      -0.4581
47     Monthly_Income_Level_Low_Income       0.4543
26                    Department_Sales       0.3634
4              EnvironmentSatisfaction      -0.3463
38          JobRole_Research Scientist      -0.3360

Archivos 'hr_churn_drivers.csv' y 'hr_data_fe_ready.csv' generados.
